In [ ]:
! pip install pymilvus[model] datasets

In [ ]:
ZILLIZDB_USERNAME = "db_f551eced4483bcc"
ZILLIZDB_PASSWORD = "Yn9%Ts4tgkDm/ta/"
ZILLIZDB_HOST = "in05-2f51803fc57ed00.serverless.gcp-us-west1.cloud.zilliz.com"
ZILLIZDB_PORT = "19531"
ZILLIZDB_TOKEN = "f41eb4a7c305fd31720650e0ea5c037dae612fc47865cad5c5edd5429c14e1e96582b2ac87c0abcd26241fa4b3611c3054573682"
ZILLIZDB_COLLECTION_NAME_DOCUMENTS = "argo_documents_hs"

In [ ]:
import pandas as pd
from datasets import load_dataset

from pymilvus import (
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
    AnnSearchRequest,
    RRFRanker,
    connections,
)

from pymilvus.model.hybrid import BGEM3EmbeddingFunction

In [ ]:
dataset = load_dataset("tasksource/esci", split="train")

dataset = dataset.select(range(5))
dataset = dataset.filter(lambda x: x["product_locale"] == "us")
dataset

In [ ]:
source_df = dataset.to_pandas()

df = source_df.drop_duplicates(
    subset=["product_text", "product_title", "product_bullet_point", "product_brand"]
)
# Drop rows with missing values
df = df.dropna(
    subset=["product_text", "product_title", "product_bullet_point", "product_brand"]
)
df.head()

In [ ]:
df["merged_text"] = df["product_title"] + "\n" + df["product_text"] + "\n" + df["product_bullet_point"]
docs = df["merged_text"].to_list()

In [ ]:
ef = BGEM3EmbeddingFunction(use_fp16=False, device="cpu")
dense_dim = ef.dim["dense"]
docs_embeddings = ef(docs)
query = "Do you have an example of a Panasonic product?"
query_embeddings = ef([query])

In [ ]:
from pymilvus import connections

connections.connect(
    uri=f'https://{ZILLIZDB_HOST}',
    token=ZILLIZDB_TOKEN
)

In [ ]:
fields = [
    # Use auto generated id as primary key
    FieldSchema(
        name="pk", dtype=DataType.VARCHAR, is_primary=True, auto_id=True, max_length=100
    ),
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=8192),
    FieldSchema(name="sparse_vector", dtype=DataType.SPARSE_FLOAT_VECTOR),
    FieldSchema(name="dense_vector", dtype=DataType.FLOAT_VECTOR, dim=dense_dim),
]
schema = CollectionSchema(fields, "")
col = Collection("sparse_dense_demo", schema)

In [ ]:
sparse_index = {"index_type": "SPARSE_INVERTED_INDEX", "metric_type": "IP"}
dense_index = {"index_type": "FLAT", "metric_type": "COSINE"}
col.create_index("sparse_vector", sparse_index)
col.create_index("dense_vector", dense_index)

In [ ]:
entities = [
    docs,
    docs_embeddings["sparse"],
    docs_embeddings["dense"],
]
col.insert(entities)

In [ ]:
col = Collection("argo_documents_hs")

In [ ]:
def query_hybrid_search(query: str):
    query_embeddings = ef([query])

    sparse_req = AnnSearchRequest(
        query_embeddings["sparse"], "sparse_vector", {"metric_type": "IP"}, limit=2
    )
    dense_req = AnnSearchRequest(
        query_embeddings["dense"], "dense_vector", {"metric_type": "COSINE"}, limit=2
    )

    res = col.hybrid_search(
        [sparse_req, dense_req], rerank=RRFRanker(), limit=2, output_fields=["content"]
    )

    return res

In [ ]:
sr = query_hybrid_search("Tín chỉ carbon là gì?")[0]

In [ ]:
sr = sr[0]

In [ ]:
sr

In [ ]:
docs = [item.get("entity").get("id") for item in sr]

In [ ]:
sr.entity